# Imports

In [6]:
import os
from importlib import reload
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
from utils.data_loader import load_FEANet_data
from model.Evaluator import Evaluator
from model.FEA_Net import FEA_Net

# Setup

In [12]:
seed = 666
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Add a hyperparameter searching step

# Load data

In [9]:
data = load_FEANet_data(percent=0.1)#snr=100

# Build the network

In [10]:
cfg = {'lr': 0.001,
        'epoch': 1,
        }
model = FEA_Net(data,cfg)

NameError: name 'tf' is not defined

# Train the network

In [ ]:
evaluator = Evaluator(model, data)
result = evaluator.run_newton()#run_trust_ncg

# Visualize training result

In [ ]:
evaluator.visualize(result)
for i in range(2):
    mat = result[9*i:9*(i+1)]
    print(mat.reshape(3,3))
    print(np.sum(mat))
print(model.wxt_ref.reshape(3,3))
print(np.sum(model.wxt_ref))
print(model.wyt_ref.reshape(3,3))
print(np.sum(model.wyt_ref))

In [ ]:
evaluator.init_solve(load=data['test_load'], omega=2/3.)
pred_i = np.zeros_like(data['test_resp'])  # data['test_resp']#
resp_ref = data['test_resp']
pred_i = evaluator.run_forward(model.trainable_var_ref, pred_i, resp_ref, max_itr=4000)
s0 = evaluator.solution

# Test the model

In [ ]:
evaluator.init_solve(load=data['test_load'], omega=2/3.)
pred_i = np.zeros_like(data['test_resp'])  # data['test_resp']#
pred_i = evaluator.run_forward(result, pred_i, resp_ref, max_itr=4000)
s1 = evaluator.solution

plt.figure()
plt.semilogy(s0['itr'], s0['loss'], label='ref')
plt.semilogy(s1['itr'], s1['loss'], label='pred')
plt.legend()
plt.show()